Consider the union of all lcms compounds across the three datasets

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
# load LCMS data
lcms_dir = '../../data/raw/lcms/'
lcms_files = [f for f in os.listdir(lcms_dir) if f.startswith('dansyl')]
# read data
dfs = [pd.read_csv(os.path.join(lcms_dir, f)) for f in lcms_files]
## Step 1: select compounds with high confidence identification (only Tier 1 & 2)
tier_dfs = [df[(df["Identification Level"] == "Tier 1") | (df["Identification Level"] == "Tier 2")].copy() for df in dfs]
# merge compound names of last dataset
dfs[2]['Compound'] = dfs[2]['Compound'].fillna(dfs[2]['Compound.1'])
tier_dfs[2]['Compound'] = tier_dfs[2]['Compound'].fillna(tier_dfs[2]['Compound.1'])
# extract compounds names
compounds = [df["Compound"].tolist() for df in tier_dfs]
print('Number of individual, high quality compounds per animal: ', [len(c) for c in compounds])
# find intersection of compounds in all 3 datasets
common_comps = list(set(compounds[0]).intersection(set(compounds[1])).intersection(set(compounds[2])))
common_comps.sort()
# find the group of total compounds -- union
union_comps = list(set(compounds[0]).union(set(compounds[1])).union(set(compounds[2])))
print('Number of total, high quality compounds (union): ', len(union_comps))
# order datasets according to union comp
data_order = []
for i in range(3):
    data_order.append(dfs[i][dfs[i]['Compound'].isin(union_comps)].copy())
    data_order[i] = data_order[i].set_index('Compound').reindex(union_comps).reset_index()
    # if not in Tier 1 or Tier 2, set to NaN
    data_order[i].loc[~data_order[i]['Compound'].isin(compounds[i]), data_order[i].columns != 'Compound'] = np.nan
# order so as to have common compounds first
data_order = [pd.concat([td[td["Compound"].isin(common_comps)], td[~td["Compound"].isin(common_comps)]]) for td in data_order]

Number of individual, high quality compounds per animal:  [241, 343, 345]
Number of total, high quality compounds (union):  482


In [4]:
# preprocessing steps
log_transform = True
# data_order_order extraction and columns renaming (specific for data_orderset of day 1)
metadata_order = [d.iloc[:,:21] for d in data_order]
data_order[0] = data_order[0].iloc[:,21:-4]
if log_transform:
    data_order[0] = data_order[0].transform(lambda x: np.log2(x))
new_columns = []
for col in data_order[0].columns:
    name = col[-6:]
    if name[0] == '_':
        name = name[1:]
    new_columns.append(name)
data_order[0].columns = new_columns

# columns renaming (specific for data_orderset of day 2)
data_order[1] = data_order[1].iloc[:,21:-3]
if log_transform:
    data_order[1] = data_order[1].transform(lambda x: np.log2(x))
new_columns = []
for col in data_order[1].columns:
    parts = col.split('.')
    if len(parts) >= 2:
        new_col = f"{parts[0]}{int(float(parts[1])/2)+1}_{0}{int(float(parts[1])%2+1)}"
    else:
        new_col = col+"1_01"  # Default to _01 if no replicate info
    new_columns.append(new_col)
data_order[1].columns = new_columns

# columns renaming (specific for data_orderset of day 3)
data_order[2] = data_order[2].iloc[:, 24:-3]
if log_transform:
    data_order[2] = data_order[2].transform(lambda x: np.log2(x))
new_columns = []
for icol in np.arange(0,data_order[2].shape[1],2):
    new_columns.append(f"{int(icol/2)+1}_01")
    new_columns.append(f"{int(icol/2)+1}_02")
data_order[2].columns = new_columns


# check for columns with abnormally high or low values across all datasets
# --> remove samples with intensities out of bounds (median ± 4×MAD)
# --> that was computed already in data_preprocessing_alignment, value is np.float64(-0.7805087050586059)

for d in data_order:
    for col in d.columns:
        if d[col].median() < -0.7805087050586059:
            d[col] = np.nan

# remove dirty technical replicates
def clean_row(row, thr=1):
    # 1) compute abs difference of each consecutive pair
    row_clean = []
    for i in range(1, len(row), 2):
        diff = np.abs(row[i] - row[i-1])
        if diff > thr:
            row_clean.append(np.nan)
        else:
            row_clean.append(np.mean(row[i-1:i+1]))
    return np.array(row_clean)

# do the cleaning
clean_data = []
for id, d in enumerate(data_order):
    clean_data.append([])
    for i in range(d.shape[0]):
        clean_data[id].append(clean_row(d.iloc[i,:].values, thr=1))
    clean_data[id] = np.array(clean_data[id])

In [5]:
np.save('../../data/processed/lcms_union_clean_data.npy', {'data_day1': clean_data[0], 'data_day2': clean_data[1], 'data_day3': clean_data[2]})